In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [51]:
TeslaR = pd.read_csv("BareTesla.csv")

In [52]:
TeslaR.head(5)

,Unnamed: 0,Date,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,appears_in,symbols,related_articles
0,1,2018-03-13,62,{'$oid': '5ab8cb373efc2c03c520c0cb'},https://www.nasdaq.com/article/can-tesla-inc-h...,"Can Tesla, Inc. Hit Its Model 3 Production Tar...",{'$date': '2018-03-13T11:05:00Z'},"By Daniel Sparks, Motley Fool",https://www.nasdaq.com/author/fool,With just a few weeks left in Tesla's (NASDAQ:...,"PersonalFinance,Stocks",TSLA,NaN
1,2,2018-03-13,370,{'$oid': '5ab8d0e43efc2c03c520cd51'},https://www.nasdaq.com/article/6-energy-stocks...,6 Energy Stocks I'd Avoid at All Costs,{'$date': '2018-03-13T06:05:00Z'},"By Travis Hoium, Motley Fool",https://www.nasdaq.com/author/fool,The energy industry has been one of the most s...,"PersonalFinance,Stocks","TTM,EV,XOM,TSLA,NRG",NaN
2,3,2018-03-12,469,{'$oid': '5ab8d1fb3efc2c03c520cfc7'},https://www.nasdaq.com/article/better-buy-tesl...,"Better Buy: Tesla, Inc. vs. Ford",{'$date': '2018-03-12T21:09:00Z'},"By Dan Caplinger, Motley Fool",https://www.nasdaq.com/author/fool,Strength in the auto industry has lifted key c...,"PersonalFinance,Stocks","TSLA,F",NaN
3,4,2018-03-13,787,{'$oid': '5ab8d5ed3efc2c03c520d8a7'},https://www.nasdaq.com/article/why-we-might-be...,Why We Might Be Hitting Peak Smartphone,{'$date': '2018-03-13T16:48:13Z'},"By Chris Hill, Motley Fool",https://www.nasdaq.com/author/fool,"In to this episode ofMarketFoolery , analyst S...","PersonalFinance,Stocks",TSLA,NaN
4,5,2018-03-13,793,{'$oid': '5ab8d61e3efc2c03c520d915'},https://www.nasdaq.com/article/nasdaq-streak-s...,"Nasdaq Streak Snapped on Tech Pullback, White ...",{'$date': '2018-03-13T16:26:43Z'},"By pmartin@sir-inc.com, Schaeffer's Research",https://www.nasdaq.com/author/schaeffers,"Stocks once again started the day off strong, ...","Investing,Options","UNH,TSLA,AAPL,DDD,EIA",NaN


In [69]:
TeslaAksje = pd.read_csv("TeslaLabelFinal1.csv")

In [70]:
TeslaAksje.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Label
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0,1
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0,0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0,0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0,0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0,0


In [55]:
TeslaR.shape

(9414, 13)

In [56]:
TeslaAksje.shape

(2917, 1)

In [71]:
#Merge datasettene
merge = TeslaR.merge(TeslaAksje, how = "inner", on = "Date")

In [72]:
merge

,Unnamed: 0,Date,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,...,symbols,related_articles,Open,High,Low,Close,Volume,Dividends,Stock Splits,Label
0,1,2018-03-13,62,{'$oid': '5ab8cb373efc2c03c520c0cb'},https://www.nasdaq.com/article/can-tesla-inc-h...,"Can Tesla, Inc. Hit Its Model 3 Production Tar...",{'$date': '2018-03-13T11:05:00Z'},"By Daniel Sparks, Motley Fool",https://www.nasdaq.com/author/fool,With just a few weeks left in Tesla's (NASDAQ:...,...,TSLA,NaN,65.722000,69.442001,65.300003,68.367996,29829000,0,0.0,1
1,2,2018-03-13,370,{'$oid': '5ab8d0e43efc2c03c520cd51'},https://www.nasdaq.com/article/6-energy-stocks...,6 Energy Stocks I'd Avoid at All Costs,{'$date': '2018-03-13T06:05:00Z'},"By Travis Hoium, Motley Fool",https://www.nasdaq.com/author/fool,The energy industry has been one of the most s...,...,"TTM,EV,XOM,TSLA,NRG",NaN,65.722000,69.442001,65.300003,68.367996,29829000,0,0.0,1
2,4,2018-03-13,787,{'$oid': '5ab8d5ed3efc2c03c520d8a7'},https://www.nasdaq.com/article/why-we-might-be...,Why We Might Be Hitting Peak Smartphone,{'$date': '2018-03-13T16:48:13Z'},"By Chris Hill, Motley Fool",https://www.nasdaq.com/author/fool,"In to this episode ofMarketFoolery , analyst S...",...,TSLA,NaN,65.722000,69.442001,65.300003,68.367996,29829000,0,0.0,1
3,5,2018-03-13,793,{'$oid': '5ab8d61e3efc2c03c520d915'},https://www.nasdaq.com/article/nasdaq-streak-s...,"Nasdaq Streak Snapped on Tech Pullback, White ...",{'$date': '2018-03-13T16:26:43Z'},"By pmartin@sir-inc.com, Schaeffer's Research",https://www.nasdaq.com/author/schaeffers,"Stocks once again started the day off strong, ...",...,"UNH,TSLA,AAPL,DDD,EIA",NaN,65.722000,69.442001,65.300003,68.367996,29829000,0,0.0,1
4,9,2018-03-13,1113,{'$oid': '5ab8d62a3efc2c03c520d933'},https://www.nasdaq.com/article/tesla-bulls-bew...,Tesla Bulls Beware: Volkswagen Wants to Be the...,{'$date': '2018-03-13T16:20:00Z'},"By Benjamin Rains, Zacks.com",https://www.nasdaq.com/author/zacks,German automotive giant Volkswagen VLKAYdetail...,...,"F,GM,TSLA,TM",NaN,65.722000,69.442001,65.300003,68.367996,29829000,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672,9390,2019-08-27,1152221,{'$oid': '5d6656d883ce455bc11af07b'},https://www.nasdaq.com/article/nio-stock-price...,Nio Stock Price Will Make a Comeback,{'$date': '2019-08-27T10:11:26Z'},"By David Moadel, InvestorPlace Media",https://www.nasdaq.com/author/investorplacemedia,"InvestorPlace - Stock Market News, Stock Advic...",...,"NIO,TSLA",[{'news_title': 'The 52-Week Highs You Never H...,43.147999,43.759998,42.405998,42.816002,27081000,0,0.0,0
8673,9397,2019-08-27,1152781,{'$oid': '5d6655f783ce455bc11ae829'},https://www.nasdaq.com/article/tesla-stock-fac...,Tesla Stock Faces More Bad News After Walmart ...,{'$date': '2019-08-27T12:43:06Z'},"By Tim Biggam, InvestorPlace Media",https://www.nasdaq.com/author/investorplacemedia,"InvestorPlace - Stock Market News, Stock Advic...",...,"TSLA,AMZN,WMT",[{'news_title': 'The 52-Week Highs You Never H...,43.147999,43.759998,42.405998,42.816002,27081000,0,0.0,0
8674,9401,2019-08-27,1153141,{'$oid': '5d665d9783ce455bc11b2f6f'},https://www.nasdaq.com/article/how-to-invest-i...,How to Invest in Self-Driving Car Stocks,{'$date': '2019-08-27T16:38:00Z'},"By John Rosevear, Motley Fool",https://www.nasdaq.com/author/fool,Investors have been buzzing about the tremendo...,...,"DPZ,TSLA,F,INTC,GOOG","[{'news_title': ""The Retail Apocalypse Isn't W...",43.147999,43.759998,42.405998,42.816002,27081000,0,0.0,0
8675,9403,2019-08-27,1153260,{'$oid': '5d665f1483ce455bc11b3dd3'},https://www.nasdaq.com/article/how-elon-musk-a...,How Elon Musk and Tesla Made Electric Cars Cool,{'$date': '2019-08-27T14:48:00Z'},"By Chris Neiger, Motley Fool",https://www.nasdaq.com/author/fool,"Believe it or not, the electric car isn't exac...",...,"TSLA,F,TM,GM","[{'news_title': ""The Retail Apocalypse Isn't W...",43.147999,43.759998,42.405998,42.816002,2

In [73]:
#Create a function to get the subjecticity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a fuction to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [74]:
#Create two new columns "Subjectivity" and "Polarity"
merge["Subjectivity"] = merge["article_content"].apply(getSubjectivity)
merge["Polarity"] = merge["article_content"].apply(getPolarity)

In [75]:
#Show the new columns in the merge data set
merge.head(3)

,Unnamed: 0,Date,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,...,Open,High,Low,Close,Volume,Dividends,Stock Splits,Label,Subjectivity,Polarity
0,1,2018-03-13,62,{'$oid': '5ab8cb373efc2c03c520c0cb'},https://www.nasdaq.com/article/can-tesla-inc-h...,"Can Tesla, Inc. Hit Its Model 3 Production Tar...",{'$date': '2018-03-13T11:05:00Z'},"By Daniel Sparks, Motley Fool",https://www.nasdaq.com/author/fool,With just a few weeks left in Tesla's (NASDAQ:...,...,65.722,69.442001,65.300003,68.367996,29829000,0,0.0,1,0.513533,0.184208
1,2,2018-03-13,370,{'$oid': '5ab8d0e43efc2c03c520cd51'},https://www.nasdaq.com/article/6-energy-stocks...,6 Energy Stocks I'd Avoid at All Costs,{'$date': '2018-03-13T06:05:00Z'},"By Travis Hoium, Motley Fool",https://www.nasdaq.com/author/fool,The energy industry has been one of the most s...,...,65.722,69.442001,65.300003,68.367996,29829000,0,0.0,1,0.512743,0.221662
2,4,2018-03-13,787,{'$oid': '5ab8d5ed3efc2c03c520d8a7'},https://www.nasdaq.com/article/why-we-might-be...,Why We Might Be Hitting Peak Smartphone,{'$date': '2018-03-13T16:48:13Z'},"By Chris Hill, Motley Fool",https://www.nasdaq.com/author/fool,"In to this episode ofMarketFoolery , analyst S...",...,65.722,69.442001,65.300003,68.367996,29829000,0,0.0,1,0.499273,0.135207


In [76]:
#Create a function to get the sentiment scores
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [77]:
#Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge["article_content"])):
    SIA = getSIA(merge["article_content"][i])
    compound.append(SIA["compound"])
    neg.append(SIA["neg"])
    neu.append(SIA["neu"])
    pos.append(SIA["pos"])

In [78]:
#Store the sentiment scores in the merge data set:
merge["Compound"] = compound
merge["Negative"] = neg
merge["Neutral"] = neu
merge["Positive"] = pos

In [79]:
#Show the merge data
merge.head(3)

,Unnamed: 0,Date,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,...,Volume,Dividends,Stock Splits,Label,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,1,2018-03-13,62,{'$oid': '5ab8cb373efc2c03c520c0cb'},https://www.nasdaq.com/article/can-tesla-inc-h...,"Can Tesla, Inc. Hit Its Model 3 Production Tar...",{'$date': '2018-03-13T11:05:00Z'},"By Daniel Sparks, Motley Fool",https://www.nasdaq.com/author/fool,With just a few weeks left in Tesla's (NASDAQ:...,...,29829000,0,0.0,1,0.513533,0.184208,0.9905,0.039,0.867,0.094
1,2,2018-03-13,370,{'$oid': '5ab8d0e43efc2c03c520cd51'},https://www.nasdaq.com/article/6-energy-stocks...,6 Energy Stocks I'd Avoid at All Costs,{'$date': '2018-03-13T06:05:00Z'},"By Travis Hoium, Motley Fool",https://www.nasdaq.com/author/fool,The energy industry has been one of the most s...,...,29829000,0,0.0,1,0.512743,0.221662,0.9975,0.068,0.802,0.130
2,4,2018-03-13,787,{'$oid': '5ab8d5ed3efc2c03c520d8a7'},https://www.nasdaq.com/article/why-we-might-be...,Why We Might Be Hitting Peak Smartphone,{'$date': '2018-03-13T16:48:13Z'},"By Chris Hill, Motley Fool",https://www.nasdaq.com/author/fool,"In to this episode ofMarketFoolery , analyst S...",...,29829000,0,0.0,1,0.499273,0.135207,0.9997,0.027,0.864,0.108


In [80]:
#Create a list of column to keep
keep_columns = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df = merge[keep_columns]
df

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,65.722000,65.300003,69.442001,29829000,0.513533,0.184208,0.9905,0.039,0.867,0.094,1
1,65.722000,65.300003,69.442001,29829000,0.512743,0.221662,0.9975,0.068,0.802,0.130,1
2,65.722000,65.300003,69.442001,29829000,0.499273,0.135207,0.9997,0.027,0.864,0.108,1
3,65.722000,65.300003,69.442001,29829000,0.463206,-0.015142,0.9501,0.056,0.850,0.094,1
4,65.722000,65.300003,69.442001,29829000,0.435281,0.023066,0.9924,0.002,0.923,0.074,1
...,...,...,...,...,...,...,...,...,...,...,...
8672,43.147999,42.405998,43.759998,27081000,0.511702,0.085723,0.9015,0.071,0.843,0.086,0
8673,43.147999,42.405998,43.759998,27081000,0.540350,0.054140,-0.9881,0.107,0.833,0.060,0
8674,43.147999,42.405998,43.759998,27081000,0.454072,0.102247,0.9997,0.037,0.873,0.090,0
8675,43.147999,42.405998,43.759998,27081000,0.469294,0.179761,0.9863,0.060,0.848,0.092,0


In [81]:
#Create the feature data set
X = df
X = np.array(X.drop(["Label"], 1))

#Create the target data set
y = np.array(df["Label"])

/var/folders/m7/01kjn8yx6d3_tjkcr1dkd9g80000gn/T/ipykernel_527/988765945.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(X.drop(["Label"], 1))


In [82]:
#Split the data into 80% trainig and 20% testing data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [83]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [84]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

array([1, 0, 1, ..., 1, 1, 1])

In [85]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [86]:
#Show the model metrics
print( classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86       922
           1       0.83      0.85      0.84       814

    accuracy                           0.85      1736
   macro avg       0.85      0.85      0.85      1736
weighted avg       0.85      0.85      0.85      1736

